* https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store

In [ ]:
import pandas as pd
import time
import re
from tqdm import tqdm
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%time df = pd.read_parquet("2019-Oct-Nov-parquet.parquet")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df[["event_time", "user_id"]].copy()

In [ ]:
df.info()

In [ ]:
%time df["event_time"] = pd.to_datetime(df["event_time"])

In [ ]:
%time df["first_event_time"] = df.groupby("user_id")["event_time"].transform("min")
df.info()

In [ ]:
df["first_event_time"] = pd.to_datetime(df["first_event_time"])

In [ ]:
%time df["cohort_days"] = (df["event_time"] - df["first_event_time"]).dt.days

In [ ]:
df.info()

In [ ]:
df["cohort_days"] = pd.to_numeric(df["cohort_days"], downcast="unsigned")
df.info()

In [ ]:
df["cohort_days"].value_counts().sort_index().plot()

In [ ]:
df.info()

In [ ]:
%time df["first_day"] = df["first_event_time"].dt.strftime("%m-%d")
df

In [ ]:
%time df["first_day"] = df["first_event_time"].astype(str).str[5:10]

In [ ]:
%time cohort_count = df.groupby(["first_day", "cohort_days"])["user_id"].nunique().unstack()

In [ ]:
%time cohort_count = df.groupby(["first_day", "cohort_days"])["user_id"].nunique().unstack()

In [ ]:
cohort_count

In [ ]:
cohort_count.style.background_gradient(axis=None).format("{:,.0f}")

In [ ]:
cohort_ratio = cohort_count.div(cohort_count[0], axis=0)
cohort_ratio

In [ ]:
sns.heatmap(cohort_ratio, cmap="Blues")

In [ ]:
df.head(2)

In [ ]:
%time df['week'] = df['event_time'].dt.isocalendar().week
%time df['first_week'] = df['first_event_time'].dt.isocalendar().week

In [ ]:
%time df['cohort_week'] = df['week'] - df['first_week'] + 1

In [ ]:
%time df_week = df[["first_week", "cohort_week", "user_id"]].copy()
df_week.shape

In [ ]:
%time cohort_week_count = df_week.groupby(["first_week", "cohort_week"])["user_id"].nunique().unstack()

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(cohort_week_count, cmap="Blues", annot=True, fmt=",.0f")